# Setup

In [ ]:
import os
import sys
import time
import json
import urllib3
import traceback

from IPython.display import clear_output
from random import choice, seed

In [ ]:
sys.path.append('../') 

In [ ]:
from src.generate_persona import generate_agent_pool_pro, generate_agent_pool_con
from src.connectors import parse_request_and_execute
from src.debate import start_debate, next_debate_round, next_debate_round_toxic

In [2]:
# need to disable verification of certificate, this removes the re-occuring warnings 
urllib3.disable_warnings()

In [ ]:
# replace with your ollama server url
url = "https://xxxxx/llm/ollama/llama3.1-405b/v1/chat/completions"

In [ ]:
# add file with authentification token
# with open('./conf/auth.txt', 'r') as file:
#    token = file.read()
token = input()
clear_output()

In [5]:
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json",
}

In [ ]:
options_dict = {
    "temperature": 0,
    "seed": 42
}
n_agents = 10

In [7]:
with open(f'./data/debate_topics.json', 'r') as fp:
    topics = json.load(fp)

# Generate Personas

In [ ]:
for topic_ix, proposition in enumerate(topics.keys()):
    topic_nr = topic_ix + 1
    
    file_path = f'./data/pool_of_{n_agents}_debate_pro_agents_for_topic_{topic_nr}.json'
    #if topic_nr == 1:
    #    continue

    if os.path.isfile(file_path):
        print(f"pro-pool for topic nr. {topic_nr} already exists at {file_path} - {proposition}")
    else:
    
        print(f"processing pro-pool for topic nr. {topic_nr} - {proposition}")
    
        start_time = time.time()

        response = parse_request_and_execute(
            generate_agent_pool_pro(proposition, n_agents), 
            url, 
            headers, 
            options_dict, 
            max_retries=5,
            delay=60
        )
    
        total_time = time.time() - start_time
        print(f"--- Topic {topic_nr}: {total_time / 60:.2f} minutes --- {proposition}")
        
        pool_of_pro_debate_agents = [
            json.loads(("{" + resp.split("}")[0].split(" (modified to) ")[0] + "}").strip())
            for resp in 
            response.json()["choices"][0]["message"]["content"].split("{")
            if "}" in resp
        ]
        
        with open(file_path, 'w') as fp:
            json.dump(pool_of_pro_debate_agents, fp)

    file_path = f'./data/pool_of_{n_agents}_debate_con_agents_for_topic_{topic_nr}.json'

    if os.path.isfile(file_path):
        print(f"con-pool for topic nr. {topic_nr} already exists at {file_path} - {proposition}")
    else:
    
        print(f"processing con-pool for topic nr. {topic_nr} - {proposition}")
    
        start_time = time.time()
    
        response = parse_request_and_execute(
            generate_agent_pool_con(proposition, n), 
            url, 
            headers, 
            options_dict, 
            max_retries=5,
            delay=60
        )
    
        total_time = time.time() - start_time
        print(f"--- Topic {topic_nr}: {total_time / 60:.2f} minutes --- {proposition}")
        
        pool_of_con_debate_agents = [
            json.loads(("{" + resp.split("}")[0].split(" (modified to) ")[0] + "}").strip())
            for resp in 
            response.json()["choices"][0]["message"]["content"].split("{")
            if "}" in resp
        ]
        
        with open(file_path, 'w') as fp:
            json.dump(pool_of_con_debate_agents, fp)
    

# Debate

In [ ]:
seed(42)
N = 1000
n = 10

for j in range(1, N+1):
    topic_ix, proposition = choice(list(enumerate(topics.keys())))
    topic_nr = topic_ix + 1

    file_path = f'./data/pool_of_{n}_debate_con_agents_for_topic_{topic_nr}.json'
    with open(file_path, 'r') as fp:
        pool_of_con_debate_agents = json.load(fp)

    file_path = f'./data/pool_of_{n}_debate_pro_agents_for_topic_{topic_nr}.json'
    with open(file_path, 'r') as fp:
        pool_of_pro_debate_agents = json.load(fp)
    
    print("discussion", j, ", proposition", proposition)

    
    for toxicity_level in ["no", "mild", "moderate", "heavy"]:
        
        debate_missing = True
        i = 0
        # initialize discussion
        while debate_missing:
            try:
                debate = start_debate(
                    proposition=proposition, 
                    topic_ix=topic_ix, 
                    topic_nr=topic_nr, 
                    pool_of_con_debate_agents=pool_of_con_debate_agents, 
                    n_con_agents=1, 
                    pool_of_pro_debate_agents=pool_of_pro_debate_agents, 
                    n_pro_agents=1
                )
                # not missing anymore
                debate_missing = False
            except Exception as e:
                print("failed:", type(e).__name__, "–", e)
            i += 1
            if i > 100:
                break
    
        # preparing discussion
        
        
        pro_or_con_toxic = choice(["pro", "con"])
        print("pro_or_con_toxic", pro_or_con_toxic)
        
        print("toxicity_level", toxicity_level)
        debate["pro_or_con_toxic"] = pro_or_con_toxic
        
        
        if toxicity_level == "no":
            i = 0
            still_arguing = True
            # running discussion
            while still_arguing:
                try:
                    debate = next_debate_round(debate)
                    # still_arguing = False
                    file_path = f'./data/toxic_and_baseline_random/discussion_{j}_round_{i}_for_topic_{topic_nr}_neither_toxic_level_{toxicity_level}.json'
                    with open(file_path, 'w') as fp:
                        json.dump(debate, fp)
                    if debate["in_alignment"]:
                        # then it would be over
                        still_arguing = False
                except Exception as e:
                    print("failed:", type(e).__name__, "–", e)
                    print(traceback.format_exc())
                    # response_content is a global variable set in next_debate_round_toxic
                    print(response_content)
                    debate["nrounds"] =  debate["nrounds"] - 1
                i += 1
                if i > 100:
                    break
        else:  
            i = 0
            still_arguing = True
            # running discussion
            while still_arguing:
                try:
                    debate = next_debate_round_toxic(debate, pro_or_con_toxic, toxicity_level=toxicity_level)
                    # still_arguing = False'
                    file_path = f'./data/toxic_and_baseline_random/discussion_{j}_round_{i}_for_topic_{topic_nr}_{pro_or_con_toxic}_toxic_level_{toxicity_level}.json'
                    with open(file_path, 'w') as fp:
                        json.dump(debate, fp)
                    if debate["in_alignment"]:
                        # then it would be over
                        still_arguing = False
                except Exception as e:
                    print("failed:", type(e).__name__, "–", e)
                    print(traceback.format_exc())
                    # response_content is a global variable set in next_debate_round_toxic
                    print(response_content)
                    debate["nrounds"] =  debate["nrounds"] - 1
                i += 1
                if i > 100:
                    break
        print("")
        


discussion 9 , proposition We should block access to social messaging networks during riots
'Getting first argument (con) --- 
